In [11]:
from random import shuffle
from IPython.display import clear_output
suit = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}
rank = {'Ace', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King'}
values = {'Ace': 11, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5, 'Six': 6, 'Seven': 7, 'Eight': 8, 'Nine': 9, 'Ten': 10, 'Jack': 10, 'Queen': 10, 'King': 10}
player_types = {'P', 'D'}

In [12]:
class Card():
    def __init__(self, suit, rank):
        self.suit = suit.capitalize()
        self.rank = rank.capitalize()
        self.value = values[rank.capitalize()]
        
    def __str__(self):
        return f'{self.rank} of {self.suit}, Value --> {self.value}'
    
    def reset_ace_val(self):
        if self.rank == 'Ace':
            self.value = 1

In [13]:
class Deck():
    def __init__(self):
        self.deck_cards = [Card(s, r) for s in suit for r in rank]
        
    def shuffle(self):
        shuffle(self.deck_cards)

In [14]:
class Player():
    def __init__(self, name, player_type, amount):
        self.name = name
        self.amount = amount
        self.total = 0
        self.bet = 0
        self.revealed = False
        if player_type in player_types:
            self.player_type = player_type
        self.player_cards = list()
    
    def add_card(self, card):
        self.player_cards.append(card)
        if self.player_type == 'P':
            self.revealed = True
            self.print_cards()
            self.calc_total()
        elif self.player_type == 'D' and self.revealed:            
            self.print_cards()
            self.calc_total()        
        else:
            self.show_card()
    
    def print_cards(self):
        print(f'Cards for the Player {self.name}')
        for c in self.player_cards:            
            print(c)
            
    def calc_total(self):
        self.total = 0
        reset_val = False
        for c in self.player_cards:
            self.total += c.value
        if self.total > 21:
            for c in self.player_cards:
                if c.rank == 'Ace' and c.value == values[c.rank]:
                    c.reset_ace_val()
                    reset_val = True
                    break
            if reset_val:
                self.calc_total()
        print(f'Player Total --> {self.total}')
        
    def show_card (self):
        if self.player_type == 'D':
            print(f'Cards for the Player {self.name}')
            print(self.player_cards[0])                  
    
    def reveal_cards(self):
        if self.player_type == 'D':
            self.print_cards()
            self.calc_total()
            self.revealed = True
            
    def clear_hand (self):
        self.player_cards.clear()
        self.revealed = False
        
    def win(self, amount):
        self.amount+=amount
        
    def set_bet (self, bet):
        self.bet = bet
        self.amount -= bet

In [15]:
class Table():
    def __init__(self, no_of_decks, dealer, player, bet):
        self.table_cards = list()
        self.discard_cards = list()
        self.player = player
        self.dealer = dealer
        self.bet = bet
        self.player.set_bet(bet)
        for i in range(no_of_decks):
            d = Deck()
            d.shuffle()
            self.table_cards.extend(d.deck_cards)
    
    def shuffle(self):
        shuffle(self.table_cards)
    
    def deal_one(self):
        return self.table_cards.pop()
    
    def reset (self):
        self.table_cards.extend(self.discard_cards)
        self.discard_cards.clear()
        self.shuffle()
        
    def clear_table(self):
        self.discard_cards.extend(self.player.player_cards)
        self.discard_cards.extend(self.dealer.player_cards)
        self.player.clear_hand()
        self.dealer.clear_hand()
        if len(self.discard_cards) >= len(self.table_cards):
            self.reset()

In [16]:
game_on = True
dealer = Player('Dealer', 'D', 1_000_000)
player = Player('Sandeep', 'P', 10_000)
new_table = Table(no_of_decks=5, dealer=dealer, player=player, bet=100)
while game_on:
    clear_output(wait=True)
    # deal two rounds
    for i in range(2):
        player.add_card(new_table.deal_one())
        dealer.add_card(new_table.deal_one())
    
    round_over = False
    while not round_over:
        if player.total == 21:
            if len(player.player_cards) == 2:
                print (f'{player.name} has BlackJack')
            dealer.reveal_cards()
            if dealer.total == 21:
                if len(dealer.player_cards) == 2:
                    print (f'{dealer.name} also has BlackJack')
                    print ('BlackJack Push!!!')
                else:
                    print ('Push!!!')
                player.win(new_table.bet)
                dealer.win(new_table.bet)
                round_over = True
            else:
                print (f'{player.name} Won!!')
                player.win(new_table.bet*2.5)
                round_over = True
        elif player.total > 21:
            print (f'{player.name} has Bust, {dealer.name} Won!!')
            dealer.win(new_table.bet*2)
            round_over = True
        else:
            play = input('Enter H for Hit or S for Stay: ')
            if play.upper() == 'H':
                player.add_card(new_table.deal_one())
            elif play.upper() == 'S':                    
                dealer.reveal_cards()
                if dealer.total == 21:
                    print (f'{dealer.name} has BlackJack')
                    dealer.win(new_table.bet*2)
                    round_over = True
                else:
                    while dealer.total < player.total:
                        dealer.add_card(new_table.deal_one())
                    if dealer.total == player.total:
                        print ('Push!!!')
                        player.win(new_table.bet)
                        dealer.win(new_table.bet)
                        round_over = True
                    elif dealer.total > 21:
                        print (f'{dealer.name} has Bust, {player.name} Won!!')
                        player.win(new_table.bet*2)
                        round_over = True
                    elif dealer.total > player.total and dealer.total <= 21:
                        print (f'{dealer.name} has Won!!')
                        dealer.win(new_table.bet*2)
                        round_over = True
                    else:
                        print (f'{player.name} has Won!!')
                        player.win(new_table.bet*2)
                        round_over = True
    new_table.clear_table()
    game_on = int(input('Do you want to Play, Enter 0 to Exit: '))
    

Cards for the Player Sandeep
Six of Clubs, Value --> 6
Player Total --> 6
Cards for the Player Dealer
Five of Spades, Value --> 5
Cards for the Player Sandeep
Six of Clubs, Value --> 6
Two of Clubs, Value --> 2
Player Total --> 8
Cards for the Player Dealer
Five of Spades, Value --> 5
Enter H for Hit or S for Stay: h
Cards for the Player Sandeep
Six of Clubs, Value --> 6
Two of Clubs, Value --> 2
Ace of Spades, Value --> 11
Player Total --> 19
Enter H for Hit or S for Stay: s
Cards for the Player Dealer
Five of Spades, Value --> 5
Two of Diamonds, Value --> 2
Player Total --> 7
Cards for the Player Dealer
Five of Spades, Value --> 5
Two of Diamonds, Value --> 2
Seven of Hearts, Value --> 7
Player Total --> 14
Cards for the Player Dealer
Five of Spades, Value --> 5
Two of Diamonds, Value --> 2
Seven of Hearts, Value --> 7
Nine of Clubs, Value --> 9
Player Total --> 23
Dealer has Bust, Sandeep Won!!
Do you want to Play, Enter 0 to Exit: 0
